# Importing modules

In [31]:
from mediawiki import MediaWiki
import regex as re
import pandas as pd
from liquipediapy import counterstrike
import numpy as np
import os
from dotenv import load_dotenv

## Using liquipediapy 

In [2]:
csgo_apy = counterstrike(f"Data_gathering({os.getenv('email')})")

In [3]:
csgo_tournaments = csgo_apy.get_tournaments('Premier')
csgo_players = csgo_apy.get_players()


## Using MediaWiki

In [4]:
csgo_media = MediaWiki(url = 'https://liquipedia.net/counterstrike/api.php',timeout = 60
                       ,user_agent=f"Data_gathering({os.getenv('email')})")


# Creating tables

In [5]:
tournaments_df = pd.DataFrame(csgo_tournaments)
tournaments_df_clean = tournaments_df
tournaments_df_clean

,tier,tournament,date,prize,teams_no,host_locaion,event_locaion,first_place,second_place
0,,BLAST Premier: World Final 2022,"Dec 14 - 18, 2022","$1,000,000",8,United Arab Emirates,Abu Dhabi,TBD,TBD
1,,BLAST Premier: Fall Finals 2022,"Nov 23 - 27, 2022","$425,000",8,Denmark,Copenhagen,TBD,TBD
2,,Elisa Masters Espoo 2022,"Nov 16 - 20, 2022","$200,000",12,Finland,Espoo,TBD,TBD
3,,Intel Extreme Masters Rio Major 2022,"Oct 31 - Nov 13, 2022","$1,000,000",24,Brazil,Rio de Janeiro,TBD,TBD
4,,ESL Pro League Season 16,"Aug 31 - Oct 02, 2022","$835,000",24,,Malta,TBD,TBD
...,...,...,...,...,...,...,...,...,...
212,,DreamHack Winter 2012,"Nov 22 - 24, 2012","$45,275.16",16,Sweden,Jönköping,NiP,VG
213,,ESWC 2012,"Oct 31 - Nov 04, 2012","$22,000",10,France,Paris,NiP,VG
214,,DreamHack Summer 2012,"Jun 16 - 20, 2012","$14,221.70",8,Sweden,Jönköping,Fnatic,Na`Vi
215,,Intel Extreme Masters VI,"Mar 06 - 10, 2012","$100,000",12,Germany,Hannover,ESC,Na`Vi


In [6]:
players_df = pd.DataFrame(csgo_players)
players_df

,id,name,country,team
0,stikle,,Albania,
1,chrissK,ChristophKröll,Austria,
2,dukiiii,DukiIvkovic,Austria,
3,kakafu,AlexanderSzymanczyk,Austria,
4,NinoZjE,AnelZukić,Austria,
...,...,...,...,...
1831,ZipZip,RikusKlue,South Africa,
1832,Potei,AlaeddineChaari,Tunisia,
1833,Remind,AdelGuermassi,Tunisia,
1834,Sheva,HatemGuermassi,Tunisia,


## Cleaning tables - Tournament

### Tier

In [7]:
tournaments_df_clean = tournaments_df.drop('tier',axis = 1)

### Date

In [8]:
tournaments_df_clean['date'] = tournaments_df['date'].str.replace('(.*-) .*, ([0-9]*)','\\1 \\2',regex = True)
tournaments_df_clean.loc[tournaments_df_clean['date'] == 'Mar\xa0??, 2020','date'] = 'Mar 15 - 2020'#tournaments_df_clean[tournaments_df_clean['date'] == 'Mar\xa0??, 2020']['date'].str.replace('[??].','15',regex = True)
tournaments_df_clean['date'] = pd.to_datetime(tournaments_df_clean['date'].str.lower(), format = '%b %d - %Y') # Dec 14 - 2022

### Prize

In [9]:
tournaments_df_clean['prize'] = tournaments_df['prize'].str.replace('\$','',regex = True)
tournaments_df_clean['prize'] = tournaments_df_clean['prize'].str.replace(',','',regex = True)
tournaments_df_clean['prize'] = pd.to_numeric(tournaments_df_clean['prize'])

### Team No

In [10]:
tournaments_df_clean['teams_no'] = pd.to_numeric(tournaments_df['teams_no'])

### Host and Event Location

In [11]:
tournaments_df_clean = tournaments_df_clean.rename({'host_locaion':'host_location',	'event_locaion':'event_location'})

In [12]:
tournaments_df_clean

,tournament,date,prize,teams_no,host_locaion,event_locaion,first_place,second_place
0,BLAST Premier: World Final 2022,2022-12-14,1000000.00,8,United Arab Emirates,Abu Dhabi,TBD,TBD
1,BLAST Premier: Fall Finals 2022,2022-11-23,425000.00,8,Denmark,Copenhagen,TBD,TBD
2,Elisa Masters Espoo 2022,2022-11-16,200000.00,12,Finland,Espoo,TBD,TBD
3,Intel Extreme Masters Rio Major 2022,2022-10-31,1000000.00,24,Brazil,Rio de Janeiro,TBD,TBD
4,ESL Pro League Season 16,2022-08-31,835000.00,24,,Malta,TBD,TBD
...,...,...,...,...,...,...,...,...
212,DreamHack Winter 2012,2012-11-22,45275.16,16,Sweden,Jönköping,NiP,VG
213,ESWC 2012,2012-10-31,22000.00,10,France,Paris,NiP,VG
214,DreamHack Summer 2012,2012-06-16,14221.70,8,Sweden,Jönköping,Fnatic,Na`Vi
215,Intel Extreme Masters VI,2012-03-06,100000.00,12,Germany,Hannover,ESC,Na`Vi


## Cleaning players table

In [13]:
players_df_clean = players_df
players_df_clean = players_df_clean.drop('team', axis = 1)

### Name

In [14]:
players_df_clean['name'] = players_df['name'].str.lower()

### id

In [15]:
players_df_clean['id'] = players_df['id'].str.lower()

In [16]:
players_df_clean

,id,name,country
0,stikle,,Albania
1,chrissk,christophkröll,Austria
2,dukiiii,dukiivkovic,Austria
3,kakafu,alexanderszymanczyk,Austria
4,ninozje,anelzukić,Austria
...,...,...,...
1831,zipzip,rikusklue,South Africa
1832,potei,alaeddinechaari,Tunisia
1833,remind,adelguermassi,Tunisia
1834,sheva,hatemguermassi,Tunisia


# Creating Tournaments 2019

In [17]:
tournaments_2019 = tournaments_df_clean[tournaments_df_clean['date'].dt.year == 2019]
tournaments_2019.reset_index(drop = True,inplace = True)
tournaments_2019

,tournament,date,prize,teams_no,host_locaion,event_locaion,first_place,second_place
0,EPICENTER 2019,2019-12-17,500000.0,8,Russia,Moscow,Vitality,mouz
1,BLAST Pro Series: Global Final 2019,2019-12-12,500000.0,4,Bahrain,Riffa,Astralis,FaZe
2,ESL Pro League Season 10 - Finals,2019-12-03,600000.0,16,Denmark,Odense,mouz,Fnatic
3,Esports Championship Series Season 8 - Finals,2019-11-28,500000.0,8,United States,Arlington,Astralis,Liquid
4,CS:GO Asia Championships 2019,2019-11-20,500000.0,8,China,Shanghai,mouz,ENCE
5,ESL Pro League Season 10 - Europe,2019-10-08,60000.0,16,United Kingdom,London,TBD,TBD
6,ESL Pro League Season 10 - Americas,2019-10-08,62000.0,16,United States,Burbank,TBD,TBD
7,Intel Extreme Masters XIV - Beijing,2019-11-07,250000.0,8,China,Beijing-Haidian,Astralis,100T
8,StarSeries & i-League CS:GO Season 8,2019-10-21,500000.0,16,Turkey,Belek,EG,Fnatic
9,DreamHack Masters Malmö 2019,2019-10-01,250000.0,16,Sweden,Malmö,Fnatic,Vitality


## Cleaning Tournaments 2019

### Removing qualifiers and local tournaments

In [18]:
tournaments_2019 = tournaments_2019[tournaments_2019['first_place'] != 'TBD']
tournaments_2019 = tournaments_2019[tournaments_2019['first_place'] != 'Windigo']

### Removing lower prizer (<250.000)

In [19]:
tournaments_2019 = tournaments_2019[tournaments_2019['prize']>250000].reset_index(drop = True)

In [20]:
tournaments_2019

,tournament,date,prize,teams_no,host_locaion,event_locaion,first_place,second_place
0,EPICENTER 2019,2019-12-17,500000.0,8,Russia,Moscow,Vitality,mouz
1,BLAST Pro Series: Global Final 2019,2019-12-12,500000.0,4,Bahrain,Riffa,Astralis,FaZe
2,ESL Pro League Season 10 - Finals,2019-12-03,600000.0,16,Denmark,Odense,mouz,Fnatic
3,Esports Championship Series Season 8 - Finals,2019-11-28,500000.0,8,United States,Arlington,Astralis,Liquid
4,CS:GO Asia Championships 2019,2019-11-20,500000.0,8,China,Shanghai,mouz,ENCE
5,StarSeries & i-League CS:GO Season 8,2019-10-21,500000.0,16,Turkey,Belek,EG,Fnatic
6,StarLadder Berlin Major 2019,2019-08-23,1000000.0,24,Germany,Berlin,Astralis,AVG
7,ESL One: Cologne 2019,2019-07-02,300000.0,16,Germany,Cologne,Liquid,Vitality
8,ESL Pro League Season 9 - Finals,2019-06-18,600000.0,16,France,Montpellier,Liquid,G2
9,Esports Championship Series Season 7 - Finals,2019-06-06,500000.0,8,United Kingdom,London,Vitality,FURIA


# Gathering more data from 2019

In [21]:
tournaments_2019

,tournament,date,prize,teams_no,host_locaion,event_locaion,first_place,second_place
0,EPICENTER 2019,2019-12-17,500000.0,8,Russia,Moscow,Vitality,mouz
1,BLAST Pro Series: Global Final 2019,2019-12-12,500000.0,4,Bahrain,Riffa,Astralis,FaZe
2,ESL Pro League Season 10 - Finals,2019-12-03,600000.0,16,Denmark,Odense,mouz,Fnatic
3,Esports Championship Series Season 8 - Finals,2019-11-28,500000.0,8,United States,Arlington,Astralis,Liquid
4,CS:GO Asia Championships 2019,2019-11-20,500000.0,8,China,Shanghai,mouz,ENCE
5,StarSeries & i-League CS:GO Season 8,2019-10-21,500000.0,16,Turkey,Belek,EG,Fnatic
6,StarLadder Berlin Major 2019,2019-08-23,1000000.0,24,Germany,Berlin,Astralis,AVG
7,ESL One: Cologne 2019,2019-07-02,300000.0,16,Germany,Cologne,Liquid,Vitality
8,ESL Pro League Season 9 - Finals,2019-06-18,600000.0,16,France,Montpellier,Liquid,G2
9,Esports Championship Series Season 7 - Finals,2019-06-06,500000.0,8,United Kingdom,London,Vitality,FURIA


## Creating the tables of competing players in 2019

In [22]:
players = []
tournaments_players = {}
for tournament in tournaments_2019['tournament']:
    lista = csgo_media.search(tournament,results = 20)
    links = ['https://liquipedia.net/counterstrike/'+link for link in lista if link[-1].isdigit() or len(re.findall('Season [0-9]$',link))>0 or len(re.findall('Final',link))>0 or len(re.findall('Cologne$',link))>0 or len(re.findall('Major',link))>0 or len(re.findall('Championship',link))>0 ]
    link_url = [link for link in links if len(re.findall('Minor',link)) == 0 and len(re.findall('Open',link)) == 0]
    tabelas = pd.read_html(link_url[0].replace(' ','_'))
    players_tournaments = []
    for tabela in tabelas:
        if (tabela.shape == (6,2) or tabela.shape == (5,2)) and np.all(tabela.columns == [0,1]):
            players.append(list(tabela[1][0:5]))
            players_tournaments.append(list(tabela[1][0:5]))
            tournaments_players[tournament.replace(' ','_').lower()] = players_tournaments
            if int(tournaments_2019[tournaments_2019['tournament'] == tournament]['teams_no']) == len(players_tournaments):
                break
    

In [23]:
players_2019 = [player.lower() for teams in players for player in teams]
players_2019 = list(dict.fromkeys(players_2019))
players_2019_df = players_df_clean[players_df_clean['id'].isin(players_2019)].reset_index(drop=True)

## Creating the conection between Players and tournament

In [24]:
dic_tournaments_players = {}
for key in tournaments_players.keys():
    dic_tournaments_players[key] = np.array(tournaments_players[key]).flatten()

df_tournaments_players = pd.DataFrame(pd.Series(dic_tournaments_players)).reset_index().explode(0)

df_tournaments_players = df_tournaments_players.rename(columns={0:'player','index':'tournament'}).reset_index(drop= True)
df_tournaments_players

,tournament,player
0,epicenter_2019,flamie
1,epicenter_2019,s1mple
2,epicenter_2019,electronic
3,epicenter_2019,Boombl4
4,epicenter_2019,GuardiaN
...,...,...
815,intel_extreme_masters_xiii_-_katowice_major_2019,zhokiNg
816,intel_extreme_masters_xiii_-_katowice_major_2019,Freeman
817,intel_extreme_masters_xiii_-_katowice_major_2019,aumaN
818,intel_extreme_masters_xiii_-_katowice_major_2019,advent


# Exporting to csv 

In [25]:
tournaments_2019.reset_index(drop = True).to_csv('data/tournaments_2019.csv',sep=';')
df_tournaments_players.reset_index(drop = True).to_csv('data/tournaments_players.csv',sep=';')
players_2019_df.reset_index(drop = True).to_csv('data/players_2019.csv',sep=';')

In [27]:
lat_long_tournaments = pd.read_csv('data/tournaments_lat_long_tableau.csv', sep =';')
lat_long_tournaments = lat_long_tournaments.rename({'Event Locaion': 'event_locaion'},axis=1)
lat_long_tournaments['event_locaion'] = lat_long_tournaments['event_locaion'].str.strip()
tournaments_2019['event_locaion'] = tournaments_2019['event_locaion'].str.strip() 

In [1]:
tournaments_2019_df_lat_long = pd.merge(tournaments_2019,lat_long_tournaments,how='left',on='event_locaion')
tournaments_2019_df_lat_long.to_csv('data/tournaments_2019_df_lat_long.csv',sep=';')

NameError: name 'pd' is not defined

In [29]:
lat_long_players = pd.read_csv('data/players_countries_lat_long_tableau.csv',sep=';')
players_2019_df_lat_long = pd.merge(players_2019_df,lat_long_players,how='left',on='country')
players_2019_df_lat_long.to_csv('data/players_2019_lat_long.csv')

In [30]:
tournaments_2019_df_lat_long

,tournament,date,prize,teams_no,host_locaion,event_locaion,first_place,second_place,Host Location,Latitude (generated),Longitude (generated)
0,EPICENTER 2019,2019-12-17,500000.0,8,Russia,Moscow,Vitality,mouz,Russia,"55,752","37,615"
1,BLAST Pro Series: Global Final 2019,2019-12-12,500000.0,4,Bahrain,Riffa,Astralis,FaZe,Bahrain,"26,14","50,57"
2,ESL Pro League Season 10 - Finals,2019-12-03,600000.0,16,Denmark,Odense,mouz,Fnatic,Denmark,"55,4","10,3833"
3,Esports Championship Series Season 8 - Finals,2019-11-28,500000.0,8,United States,Arlington,Astralis,Liquid,United States,"42,0813","-87,9801"
4,CS:GO Asia Championships 2019,2019-11-20,500000.0,8,China,Shanghai,mouz,ENCE,China,"31,1667","121,4667"
5,StarSeries & i-League CS:GO Season 8,2019-10-21,500000.0,16,Turkey,Belek,EG,Fnatic,Turkey,"36,862","31,055"
6,StarLadder Berlin Major 2019,2019-08-23,1000000.0,24,Germany,Berlin,Astralis,AVG,Germany,"52,5167","13,3833"
7,ESL One: Cologne 2019,2019-07-02,300000.0,16,Germany,Cologne,Liquid,Vitality,Germany,"50,9422","6,9578"
8,ESL Pro League Season 9 - Finals,2019-06-18,600000.0,16,France,Montpellier,Liquid,G2,France,"43,6109","3,8772"
9,Esports Championship Series Season 7 - Finals,2019-06-06,500000.0,8,United Kingdom,London,Vitality,FURIA,United Kingdom,"51,5073","-0,1276"
